# Calculate Monthly Anomaly and Nino34 Index Using CDO


In this notebook we demonstrate how to calculate monthly anomoly and Nino34 index using CDO:

- Look at the information contained in our example file
- Visualise the data using ncview 
- Calculate monthly anomaly
- Calculate Nino34 Climate Index

This example uses the Coupled Model Intercomparison Project (CMIP6) collections. 

---
inspired by the notebook in https://github.com/NCI-data-analysis-platform/climate-cmip.git
- Original Authors: NCI Virtual Research Environment Team
- Keywords: CMIP, CDO, monthly anomaly, Nino34 Index
- Create Date: 2019-Oct; Update Date: 2021-Feb
---
- Adapted to DKRZ env: S. Kindermann: 2022-Aug

This notebook is licenced under the [Creative Commons Attribution 4.0 International license](https://creativecommons.org/licenses/by/4.0/)

## What is CDO?

CDO stands for "Climate Data Operators". CDO is a collection of command line operators to manipulate and analyse Climate and NWP model Data. It support the following data formats: GRIB 1/2, netCDF 3/4, SERVICE, EXTRA and IEG. There are more than 600 operators available. See [CDO's homepage](https://code.mpimet.mpg.de/projects/cdo) for more information about this library.

To load the CDO module on the interactive shell at DKRZ, run:

``` $ module load cdo```

In [1]:
#!module load cdo

### List the path of the CMIP6 data

Here we will be using an example CMIP6 file that includes the near surface temperature from the 20th century all-forcing historical simulation based on the CESM2.0 model of NCAR. 

In [2]:
! ls /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc

/pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc


### Have a look at the data 

**CDO Basic usage:** 
`$ cdo sinfo filename | head -xx`

**head** displays only onely the first lines in the termial.

In [1]:
!cdo sinfo /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc | head -15

   File format : NetCDF4 zip
    -1 : Institut Source   T Steptype Levels Num    Points Num Dtype : Parameter ID
     1 : unknown  CESM2    v instant       1   1     55296   1  F32z : -1            
   Grid coordinates :
     1 : lonlat                   : points=55296 (288x192)
                              lon : 0 to 358.75 by 1.25 degrees_east  circular
                              lat : -90 to 90 by 0.9424084 degrees_north
                        available : cellbounds
   Vertical coordinates :
     1 : surface                  : levels=1
   Time coordinate :
                             time : 1980 steps
     RefTime =  0001-01-01 00:00:00  Units = days  Calendar = 365_day  Bounds = true
  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss  YYYY-MM-DD hh:mm:ss
  1850-01-15 12:00:00  1850-02-14 00:00:00  1850-03-15 12:00:00  1850-04-15 00:00:00


### Let's see which years this file includes

We can use the function **showyear** to display all the years in this file

**Basic usage:**

```
$ cdo showyear <file>
```


In [2]:
!cdo showyear /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc

 1850 1851 1852 1853 1854 1855 1856 1857 1858 1859 1860 1861 1862 1863 1864 1865 1866 1867 1868 1869
 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889
 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909
 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929
 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949
 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969
 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989
 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012 2013 2014
cdo    showyear: Processed 1 variable over 1980 timesteps [0.17s 50MB].


### Calculate monthly anomaly 

In this exercise, we are going to calculate the monthly anomaly between 1971 and 2000. We will:

1. Choose climatology years 1971-2000 
2. Calculate the monthly average during 1971-2000
3. Subtract the monthly average from the original file to obtain the difference, i.e., anomaly.

We need three functions to implement this procedure:

 
```
$ cdo selyear, year_lower, year_upper <input.nc> <output.nc> 

```

``selyear`` selects the years of interest and generates a new file.


```
$ cdo ymonmean <input.nc> <output.nc> 

```

`ymonmean` calculates the mean monthly values from a multi-year monthly timeseries.


```
$ cdo ymonsub <input1.nc> <input2.nc> <output.nc> 

```

`ymonsub` calculates the difference between two input files and saves this difference as an output file.

However, if we do the three steps above we will generate unnecessary intermediate files. CDO allows for the piping of multiple commands so we can chain together the commands above:

```
$ cdo ymonsub <input.nc> -ymonmean -selyear,1971,2000 <input2.nc> output.nc

```


In [3]:
### create output directory if it doesn't already exist
import os
outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
### run our cdo command
!cdo ymonsub /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc -ymonmean -selyear,1971/2000 /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc ./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc

cdo(1) ymonmean: Process started
cdo(2) selyear: Process started
cdo(2) selyear: Processed 19906560 values from 1 variable over 1980 timesteps.
cdo(1) ymonmean: Processed 19906560 values from 1 variable over 360 timesteps.
cdo    ymonsub: Processed 110149632 values from 2 variables over 1992 timesteps [2.30s 88MB].


### Let's have a look at the monthly anomaly

In [4]:
!ncdump -h ./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc

netcdf tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly {
dimensions:
	time = UNLIMITED ; // (1980 currently)
	bnds = 2 ;
	lon = 288 ;
	lat = 192 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 0001-01-01 00:00:00" ;
		time:calendar = "365_day" ;
		time:axis = "T" ;
	double time_bnds(time, bnds) ;
	double lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
	double lon_bnds(lon, bnds) ;
	double lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
	double lat_bnds(lat, bnds) ;
	float tas(time, lat, lon) ;
		tas:standard_name = "air_temperature" ;
		tas:long_name = "Near-Surface Air Temperature" ;
		tas:units = "K" ;
		tas:_FillValue = 1.e+20f ;
		tas:missing_value = 1.e+20f ;
		tas:cell_measures = "

### Calculate Nino34 Index using monthly anomaly

There are several indices used to monitor the tropical Pacific, all of which are based on SST anomalies averaged across a given region.  Usually the anomalies are computed relative to a base period of 30 years.  The Niño 3.4 index and the Oceanic Niño Index (ONI) are the most commonly used indices to define El Niño and La Niña events. See more about [sea surface tempreture indices](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni).

To calcuate Nino34 Index, we need to choose the Nino34 region with lat/lon bbox = (-170,-120,-5,5)

In [5]:
!cdo sellonlatbox,-170,-120,-5,5 ./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_197101-200012_anomaly.nc ./output/Nino34Region.nc

cdo    sellonlatbox: Processed 109486080 values from 1 variable over 1980 timesteps [0.34s 50MB].


Then use function **fldavg** to calculate the field average and save the result as the Nino34 index.

In [6]:
!cdo fldavg ./output/Nino34Region.nc ./output/Nino34Idx.nc

cdo    fldavg:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 91cdo    fldavg: Processed 811800 values from 1 variable over 1980 timesteps [0.19s 50MB].


### Summary

In this example, we show how to use CDO command line operators to calculate monthly anomaly and Nino34 Climate Index.

## Reference


https://code.mpimet.mpg.de/projects/cdo/embedded/cdo.pdf